# Select File


In [ ]:
import sys
sys.path.append('../')
from GeoToolkit.Mag import Mag, ProblemSetter, MathUtils, Simulator, DataIO
from tkinter import *
import tkinter as tk
import tkinter.filedialog
from tkinter.filedialog import askdirectory
from tkinter.filedialog import askopenfilename
window = Tk() # user input window
Tk().withdraw()
MyText= StringVar()

def DisplayDir(Var):
    feedback = askopenfilename()
    Var.set(feedback)

Button(window, text='Browse File', command=DisplayDir(MyText)).pack()
Entry(window, textvariable = MyText).pack()
Button(window, text='OK', command=window.destroy).pack()
mainloop()

# Input Grid File

In [ ]:
from GeoToolkit.Mag.DataIO import loadGRDFile
dataGrid=loadGRDFile(MyText.get())
#dataGrid = DataIO.loadGeoTiffFile("./assets/Synthetic/DataGrid.tiff", plotIt=False)
#selection = Simulator.setDataExtentWidget(dataGrid)
#display(selection)


In [ ]:
from GeoToolkit.Mag.DataIO import loadGeoTiffFile
dataTiff=loadGeoTiffFile(MyText.get(),plotIt=False) 
selection = Simulator.setDataExtentWidget(dataTiff)
display(selection)


In [ ]:
import rasterio
import rasterio.features
import rasterio.warp
with rasterio.open(dataGrid) as dataset:
# Read the dataset's valid data mask as a ndarray.
    mask = dataset.dataset_mask()
# Extract feature shapes and values from the array.
for geom, val in rasterio.features.shapes(
    mask, transform= dataset.transform):
    # Transform shapes from the dataset's own coordinate
    # reference system to CRS84 (EPSG:4326).
    geom = rasterio.warp.transform_geom(
       dataset.crs, 'EPSG:3578', geom, precision=6)
# Print GeoJSON shapes to stdout.
print(geom)

# test2

In [ ]:
import rasterio
from affine import Affine

xmin = -88.23
xmax = -88.1
ymin = 20.64
ymax = 20.72

def window_from_extent(xmin, xmax, ymin, ymax, aff):
    col_start, row_start = ~aff * (xmin, ymax)
    col_stop, row_stop = ~aff * (xmax, ymin)
    return ((int(row_start), int(row_stop)), (int(col_start), int(col_stop)))

with rasterio.open('/Output/MyGeoTiff') as src:
    aff = src.affine
    meta = src.meta.copy()
    window = window_from_extent(xmin, xmax, ymin, ymax, aff)
    # Read croped array
    arr = src.read(1, window=window)
    # Update dataset metadata (if you need it)
    meta.update(height = window[0][1] - window[0][0],
                width = window[1][1] - window[1][0],
                affine = src.window_transform(window))
    meta.pop('transform', None)
    

# Low Resolution

In [ ]:
# The grid of raster values can be accessed as a numpy array and plotted:
with rasterio.open(dataGrid) as src:
   oviews = src.overviews(1) # list of overviews from biggest to smallest
   oview = oviews[-1] # let's look at the smallest thumbnail
   print('Decimation factor= {}'.format(oview))
   # NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
   thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

print('array type: ',type(thumbnail))
print(thumbnail)

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview - Band 4 {}'.format(thumbnail.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

# 

# Save File

In [ ]:
# button to save as grid.result
from tkinter.filedialog import askdirectory
from tkinter.filedialog import asksaveasfile
window = Tk() # user input window

tkinter.filedialog.asksaveasfile(selection.result)
def saveBox(
            self,
            title=None,
            fileName=None,
            dirName=None,
            fileExt=".grd",
            fileTypes=None,
            asFile=False):
        self.topLevel.update_idletasks()
        if fileTypes is None:
            fileTypes = [('all files', '*.*'), ('grid files', '.grd'),('GeoTiff files'),('.tiff')]
        # define options for opening
        options = {}
        options['defaultextension'] = fileExt
        options['filetypes'] = fileTypes
        options['initialdir'] = dirName
        options['initialfile'] = fileName
        options['title'] = title

        if asFile:
            return filedialog.asksaveasfile(mode='w', **options)
        # will return "" if cancelled
        else:
            return filedialog.asksaveasfilename(**options) 